First of all note that since $x = r\cos\theta$, $y=r\sin\theta$, from which  $x^2 + y^2 = r^2$ and $\displaystyle \tan \theta = \frac{y}{x}$, we have the following:

\begin{alignat*}{2}
\frac{\partial r}{\partial x} &= \cos \theta, &\qquad
\frac{\partial r}{\partial y} &= \sin \theta, \\
\frac{\partial \theta}{\partial x} &= - \frac{\sin \theta}{r}, &
\frac{\partial \theta}{\partial y} &= \frac{\cos\theta}{r}.
\end{alignat*}

In [1]:
import sympy as sym

r, theta = sym.symbols('r, theta')

delr_delx = sym.cos(theta)

delr_dely = sym.sin(theta)

deltheta_delx = -sym.sin(theta)/r

deltheta_dely = sym.cos(theta)/r

For a generic function, $f(r,\theta)$, we have the following relations:

\begin{align*}
\frac{\partial f}{\partial x} &= \frac{\partial f}{\partial r}\frac{\partial r}{\partial x} + \frac{\partial f}{\partial \theta}\frac{\partial \theta}{\partial x} = \frac{\partial f}{\partial r}\cos\theta - \frac{\partial f}{\partial \theta}\frac{\sin \theta}{r} \\
\frac{\partial f}{\partial y} &= \frac{\partial f}{\partial r}\frac{\partial r}{\partial y} + \frac{\partial f}{\partial \theta}\frac{\partial \theta}{\partial y} = \frac{\partial f}{\partial r} \sin\theta + \frac{\partial f}{\partial \theta}\frac{\cos \theta}{r}.
\end{align*}


We define the following Python functions to make these derivatives operable on any function of $r$ and $\theta$:

In [2]:
def del_delx(f):
    return delr_delx*sym.diff(f,r) + deltheta_delx*sym.diff(f,theta)

def del_dely(f):
    return delr_dely*sym.diff(f,r) + deltheta_dely*sym.diff(f,theta)

Define the rotation matrix:

In [3]:
Q = sym.Matrix([[sym.cos(theta), sym.sin(theta)],[-sym.sin(theta), sym.cos(theta)]])
display(Q)

Matrix([
[ cos(theta), sin(theta)],
[-sin(theta), cos(theta)]])

Consider the displacement vector $\mathbf{U}_{\rm pol} = [u(r,\theta) \;\; v(r,\theta)]^{\sf T}$ in polar coordinates. 

In [4]:
u = sym.Function('u')(r,theta)
v = sym.Function('v')(r,theta)
U_pol = sym.Matrix([u, v])
display(U_pol)

Matrix([
[u(r, theta)],
[v(r, theta)]])

We transform it to the Cartesian system using $[\mathbf{U}_{\rm rec}] = [Q]^{\sf T} [\mathbf{U}_{\rm pol}]$.

In [5]:
U_rec = (Q.T)*U_pol 
u_rec = U_rec[0]
v_rec = U_rec[1]
display(u_rec, v_rec)

u(r, theta)*cos(theta) - v(r, theta)*sin(theta)

u(r, theta)*sin(theta) + v(r, theta)*cos(theta)

$\newcommand{\urec}{u_{\rm rec}}$
$\newcommand{\vrec}{v_{\rm rec}}$
$\newcommand{\pd}{\partial}$
$\newcommand{\ep}{\varepsilon}$

We set up the strain tensor in the Cartesian system as $\displaystyle \varepsilon_{\rm rec} = \begin{bmatrix} \ep_{xx} & \ep_{xy} \\ \ep_{xy} & \ep_{yy} \end{bmatrix}$, 

where

$\displaystyle \ep_{xx} = \frac{\pd \urec}{\pd x}$,

$\displaystyle \ep_{xy} = \frac{1}{2} \left( \frac{\pd \urec}{\pd y} + \frac{\pd \vrec}{\pd x} \right)$,

$\displaystyle \ep_{yy} = \frac{\pd \vrec}{\pd y}$

We set up these strain-displacement relations using the Python functions defined earlier.

In [6]:
ep_xx = del_delx(u_rec).simplify()
ep_yy = del_dely(v_rec).simplify()
ep_xy = (sym.Rational(1,2)*(del_dely(u_rec) + del_delx(v_rec)))

ep_rec = sym.Matrix([[ep_xx, ep_xy],[ep_xy, ep_yy]])

Finally we transform from Cartesian to polar coordinates using $\displaystyle [\ep_{\rm pol}] = [Q][\ep_{\rm rec}][Q]^{\sf T}$:

In [7]:
ep_pol = (Q)*ep_rec*(Q.T)

We obtain the different components $\ep_{rr}$, $\ep_{r\theta}$, and $\ep_{\theta\theta}$ as follows:

In [8]:
ep_rr = ep_pol[0,0].simplify()
ep_rt = ep_pol[0,1].simplify()
ep_tt = ep_pol[1,1].simplify()

display(ep_rr, ep_rt, ep_tt)

Derivative(u(r, theta), r)

(r*Derivative(v(r, theta), r) - v(r, theta) + Derivative(u(r, theta), theta))/(2*r)

(u(r, theta) + Derivative(v(r, theta), theta))/r